In [1]:
from rdflib import Graph
from rdflib import RDF
from rdflib import URIRef
from rdflib import Literal
import networkx as nx
import matplotlib.pyplot as plt
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from queries import QUERY_AI, QUERY_FILM, QUERY_PERSON
from paths_to_examples import REPO_BOOKS, REPO_PEOPLE, WIKIDATA, GPT_EX, GPT_SUBCLASSES
from SPARQLWrapper import SPARQLWrapper, RDF, JSON
import random


DEBUG = True  # for debug prints
PRINTGRAPH = False # for showing the networkX graph
LOCAL = False # for choosing local ttl file path or SPARQL endpoint 

def debug_print(*args, **kwargs):
    if DEBUG:
        print(*args, **kwargs)


# printing all elements of a set
def print_set(set):
    if DEBUG:
        for i in set:
            print(i)

def show_graph(G):
    if PRINTGRAPH:
        # calculating positions for nodes
        pos = nx.spring_layout(G, k=0.5, iterations=50)

        # printing nodes and edges
        plt.figure(figsize=(12, 8))
        nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=1500, font_size=10, font_weight="bold", arrows=True)

        # printing labels for the edges
        edge_labels = nx.get_edge_attributes(G, 'label')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')

        plt.title("RDF Graph")
        plt.axis("off")
        plt.show()

        print("FINISHED PRINTING GRAPH\n\n") 

def send_query(query, sparql, format):
    sparql.setQuery(query)

    try: 
        #print("Return Format:", format)
        sparql.setReturnFormat(format)
        results = sparql.query()
        results = results.convert()
        
        #triples = results.convert() # this converts directly to an RDFlib Graph object

        # add triples to graph
        #g += triples
    except Exception as e:
        print("Query fehlgeschlagen:", e)   

    return results    

# Calculating all paths from one root node (start node) for a sparql endpoint
def find_all_paths_endpoint(endpoint_url, default_graph, literals, start_node):
    neighbors_cache = {}  # global oder in der Funktion definiert

    # helperfunction for finding neighbors of a node (triple=(node, pred, neighbor))
    def get_neighbors(node):
        
        node_str = str(node)

        # 1. Prüfen, ob der Node bereits im Cache ist
        if node_str in neighbors_cache:
            # Debug-Ausgabe
            #print(f"Cache-Treffer für {node_str}")
            return neighbors_cache[node_str]
 
        if (len(str(node)) == 0): #or (not (node.startswith("http://") or node.startswith("https://"))):
            return []  # Literale und Blank Nodes haben keine gültigen SPARQL-Patterns

        #print("\n Searching for neighbors of: " + str(node) + " will start.")

        # wollte construct verwenden, damit das mit den blank nodes gelöst wird
        query = f"""
        CONSTRUCT {{
            <{node}> ?p ?next .
        }} WHERE {{
            <{node}> ?p ?next .
        }}
        """

        query = f"""
        SELECT DISTINCT ?next WHERE {{
            <{node}> ?p ?next .
        }}
        """

        
        #sparql.setQuery(query)
        #sparql.setReturnFormat(JSON)

        results = send_query(query, sparql, JSON)
       
        # Alle Objekte der Tripel extrahieren
        neighbors = []
        for binding in results["results"]["bindings"]:
            next_obj = binding["next"]
            value = next_obj["value"]
            value_type = next_obj["type"]  # 'uri', 'literal', 'bnode'
            #if value.startswith("POLYGO"):
             #   print("Value Type: " + value_type)

            if value_type == "literal" or value_type == "typed-literal":
                lang_tag = next_obj.get("xml:lang")
                datatype = next_obj.get("datatype")
                
                if lang_tag:  # Sprach-Tag hinzufügen
                    #if value.startswith("POLYGO"):
                    #    print("lang_tag: " + lang_tag)
                    value = f"{value}@{lang_tag}"
                elif datatype:  # Datentyp hinzufügen (optional, für Vollständigkeit)
                   # if value.startswith("POLYGO"):
                    #    print("datatype: " + datatype)
                    value = f"{value}^^{datatype}"
            
            if value_type != "literal" and value_type != "typed-literal" and value_type != "uri" and value_type != "iri" and value_type != "bnode":
                print("OTHER VALUE TYPE: " + value_type)

            neighbors.append((value, value_type))
            #neighbors.append(binding["next"]["value"])
            #if str(node) == "https://nfdi.fiz-karlsruhe.de/ontology/MIME-Type":
             #   value_type = binding["next"]["type"]  # <--- Hier bekommst du den Typ: uri, literal, bnode
              #  value = binding["next"]["value"]
               # # Debug-Output
                #print(f"  Nachbar gefunden: {value} (Typ: {value_type})")    

        neighbors_cache[node_str] = neighbors

        return neighbors
    
    def dfs(path, node, node_type="uri"):
        global num_paths
        global abs_depth
        global max_depth

        # skip node if it already is in path (avoiding cycles)
        if node in path:
            return
        
        # add current node to path
        path.append(node)
        
        neighbors = list(get_neighbors(node)) if (node_type != "literal" and node_type != "typed-literal" and node not in literals) else []

        if not neighbors:
            paths.append(list(path))
            num_paths += 1
            abs_depth += (len(path) - 1)
            max_depth = max(max_depth, len(path) - 1)
        else:
            for neighbor, neighbor_type in neighbors:
                #print("\node: " + str(node) + " - neighbor: " + str(neighbor))
                # if node is literal -> it does not have any neighbors -> path is finished
                #if neighbor_type == "literal" or (neighbor in literals):
                if neighbor in literals:
                    #print("Ist in Literals: " +  neighbor)
                #if neighbor in literals:
                    path.append(neighbor)
                    #print("Found Path: ", path)
                    paths.append(list(path))
                    num_paths += 1
                    abs_depth += len(path) - 1
                    max_depth = max(max_depth, len(path) - 1)
                    # remove node from the path to find next path
                    path.pop()
                else:
                    dfs(path, neighbor, neighbor_type)

        # remove node from the path to find next path
        path.pop()

    sparql = SPARQLWrapper(endpoint_url)
    sparql.addDefaultGraph(default_graph)
    #sparql.setTimeout(60)  # 120 Sekunden Timeout

    # list which stores all paths
    paths = []

    dfs([], start_node, node_type="uri")

    return paths


In [2]:
from concurrent.futures import ThreadPoolExecutor
import sys

num_paths = 0
abs_depth = 0
max_depth = 0

endpoint_url = "https://data.europa.eu/sparql"
default_graph = "http://data.europa.eu/88u/dataset/0800af55-8e56-49a0-8986-aa55151d0440"
endpoint_url = "https://lov.linkeddata.es/dataset/lov/sparql"
#endpoint_url = "https://sparql.europeana.eu/"
#endpoint_url = "https://nfdi4culture.de/sparql"

endpoint_url = "https://lov.linkeddata.es/dataset/lov/sparql"
default_graph = "http://purl.org/cwmo/#"
default_graph = "https://w3id.org/ecfo"
default_graph = "http://vocab.deri.ie/csp"
default_graph = "http://purl.org/wf4ever/ro"

#endpoint_url = "https://data.europa.eu/sparql"
#default_graph = "http://data.europa.eu/88u/dataset/0800af55-8e56-49a0-8986-aa55151d0440"

#default_graph = "http://def.seegrid.csiro.au/isotc211/iso19156/2011/observation"

sparql = SPARQLWrapper(endpoint_url)
sparql.addDefaultGraph(default_graph)

print("STARTING GETTING LITERALS")

query_literals = """
SELECT DISTINCT ?literal
WHERE {
?s ?p ?literal .
FILTER(isLiteral(?literal))
}
"""

results = send_query(query_literals, sparql, JSON)

#count = results["results"]["bindings"][0]["literalCount"]["value"]
#print("Anzahl der Literale:", count)

literals = set()
#debug_print("Literals:")
i = 0
for res in results["results"]["bindings"]:
    #literal_value = res["literal"]["value"]
    #lang_tag = res["literal"].get("xml:lang", None)
    ## if literal has a tag
    #if lang_tag:
    #    literals.add(f"{literal_value}@{lang_tag}")
    #    print(f"{literal_value}@{lang_tag}")
    #else:
    #    literals.add(literal_value)
    #    print(literal_value)
    literal_value = res["literal"]["value"]
    lang_tag = res["literal"].get("xml:lang")
    datatype = res["literal"].get("datatype")

    if lang_tag:
        literal_value = f"{literal_value}@{lang_tag}"
    elif datatype:
        literal_value = f"{literal_value}^^{datatype}"
    #else:
     #   literal_value = literal_value

    literals.add(literal_value)
    #print(literal_value)



print(str(len(literals)) + " Literals existing")

print("FINISHED GETTING LITERALS\n")

#sys.exit()

print("STARTING CALCULATING PATHS")


query_roots = """
SELECT DISTINCT ?root
WHERE {
    ?root ?p ?o .
    FILTER NOT EXISTS {
        ?s ?p2 ?root .
    }
}
"""

results = send_query(query_roots, sparql, JSON)

#debug_print((results))
root_nodes = set()
#debug_print("Root Nodes:")
"""i = 0

# list of paths for all root nodes
all_paths = {}

for res in results["results"]["bindings"]:
    # calculating all paths from root node
   
    str_root = str(res["root"]["value"])
    debug_print("Starting searching for paths with root node: " + str_root)
    all_paths[str_root] = find_all_paths_endpoint(endpoint_url, default_graph, literals, str_root)

    #debug_print(res["root"]["value"])
    
    i += 1"""

for res in results["results"]["bindings"]:
    str_root = str(res["root"]["value"])
    root_nodes.add(str_root)

# Anzahl der Root-Nodes ausgeben
print(f"Number of root nodes: {len(root_nodes)}")

# Dictionary für alle Pfade
all_paths = {}

# Jetzt alle Pfade für die gespeicherten Root-Nodes berechnen
for str_root in root_nodes:
    debug_print("Calculating paths for root node: " + str_root)
    all_paths[str_root] = find_all_paths_endpoint(endpoint_url, default_graph, literals, str_root)


print("FINISHED CALCULATING PATHS\n")

# Output found paths
print("FOUND PATHS:")
for root_node, paths in all_paths.items():
    for path in paths:
        debug_print(path)
        debug_print("Path length = " + str(len(path)-1))

print("RESULTS:")

print("-Number of Paths: " + str(num_paths))
print("-Absolute depth: " + str(abs_depth))

# TODO: num_paths > 0
avg_depth = abs_depth / num_paths

print("-Average depth: " + str(avg_depth))
print("-Maximal depth: " + str(max_depth))

STARTING GETTING LITERALS
12 Literals existing
FINISHED GETTING LITERALS

STARTING CALCULATING PATHS
Number of root nodes: 3
Calculating paths for root node: http://purl.org/wf4ever/ro
Calculating paths for root node: http://purl.org/wf4ever/ro#AggregatedAnnotation
Calculating paths for root node: http://purl.org/wf4ever/ro#annotatesAggregatedResource
FINISHED CALCULATING PATHS

FOUND PATHS:
['http://purl.org/wf4ever/ro', 'This ontology shows how AO and ORE ontologies can be used together to define a ResearchObject. This ontology is further customized by the wf4ever ontology.@en']
Path length = 1
['http://purl.org/wf4ever/ro', 'http://www.w3.org/2002/07/owl#Ontology']
Path length = 1
['http://purl.org/wf4ever/ro', '0.1.1^^http://www.w3.org/2001/XMLSchema#string']
Path length = 1
['http://purl.org/wf4ever/ro', 'http://xmlns.com/foaf/0.1/']
Path length = 1
['http://purl.org/wf4ever/ro', 'http://purl.org/NET/dc_owl2dl/terms_od']
Path length = 1
['http://purl.org/wf4ever/ro', 'http://purl.

In [ ]:
# Calculate Tangledness 
# source 73 - page 4
# tangledness = mean number of classes with more than 1 direct ancestor, so two primitive 
# measurements (number of classes and number of direct ancestors) are used for computing the metric 

# Select number of classes in graph 
query_classes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT (COUNT(DISTINCT ?class) AS ?num_classes)
WHERE {
  {
    # 1. explicitly/implicitly used RDF classes
    # explicitly: ?class a owl:Class . or ?class a rdfs:Class .
    # implicitly: ?any rdf:type ?class . (includes also explicitly used classes)

    ?any rdf:type ?class .
  }
  UNION
  {
    # 2. subclasses
    ?class rdfs:subClassOf ?any .
  }
  UNION
  {
    # 3. superclasses
    ?any rdfs:subClassOf ?class .
  }
  UNION
  {
    # 4. classes used with owl:equivalentClass
    { ?class owl:equivalentClass ?any . }
    UNION
    { ?any owl:equivalentClass ?class . }
  }
  UNION
  {
    # 5. OWL restriction classes
    ?class a owl:Restriction .
  }
  UNION
  {
    # 6. complex classes with using unionOf, intersectionOf etc.
    ?class owl:unionOf|owl:intersectionOf|owl:complementOf|owl:oneOf ?list .
  }
  UNION
  {
    # 7. OWL hasValue restrictions
    ?class owl:hasValue ?val .
  }
}"""

results = send_query(query_classes, sparql, JSON)

num_classes = 0

for binding in results["results"]["bindings"]:
    num_classes = int(binding["num_classes"]["value"])
    print("Number of classes in graph: " + str(num_classes))

# Select number of classes with more than one ingoing isA arc (Dr. Jovanovik said I should use is-a)
# source 37 - page 3 says the same, but look at query_var2
query_var1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (COUNT(?class) AS ?tangledCount)
WHERE
{
  SELECT ?class (COUNT(?s) AS ?numIngoing)
  WHERE {
    { ?s rdf:type ?class . }
  }
  GROUP BY ?class
  HAVING (COUNT(?s) > 1) # problem here was: i used ?numIngoing instead of COUNT(?s)
}
"""

# Select number of classes with more than one superclass (source 73 says I should use this query)
query_var2 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (COUNT(?class) AS ?tangledCount)
WHERE {
  SELECT ?class (COUNT(?super) AS ?numSupers)
  WHERE {
    ?class rdfs:subClassOf ?super .
  }
  GROUP BY ?class
  HAVING (COUNT(?super) > 1)
}
"""

results = send_query(query_var2, sparql, JSON)

t = 0

for binding in results["results"]["bindings"]:
    t = int(binding["tangledCount"]["value"])
    print("Number of classes with more than one superclass: " + str(t))

print("\nRESULT:")

if num_classes > 0:
  # source 37 says num_classes / t
  # source 73 says denominator and numerator should be switched -> t / num_classes
  tangledness = t / num_classes
  print("-Tangledness: " + str(tangledness))
else:
  # TODO
   print("-Tangledness is INF")


Number of classes in graph: 33
Number of classes with more than one superclass: 5

RESULT:
-Tangledness: 0.15151515151515152


In [4]:
# Degree Distribution (Formula in: source 37, page 7)
# nG...number of nodes in gaph
# nE...number of edges in graph

print("STARTING CALCULATING DEGREE DISTRIBUTION")

# Calculating nE
query_nE = """
SELECT (COUNT(*) AS ?tripleCount)
WHERE {
  ?s ?p ?o .
}
"""


results = send_query(query_nE, sparql, JSON)


#for row in results:
    #print("Number of edges in graph: " + str(row['tripleCount']))
   # nE = int(row['tripleCount'])
#print(results)
nE = int(results["results"]["bindings"][0]["tripleCount"]["value"])
print("Number of edges in graph: ", str(nE))

# Calculating nG
query_nG = """
SELECT (COUNT(DISTINCT ?node) AS ?nodeCount)
WHERE {
  {
    SELECT ?node WHERE {
      { ?node ?p1 ?o }       
      UNION
      { ?s ?p2 ?node }       
    }
  }
}
"""

results = send_query(query_nG, sparql, JSON)

nG = int(results["results"]["bindings"][0]["nodeCount"]["value"])
print("Number of nodes in graph: ", str(nG))

# Calculating degree for every node in graph
query_degrees = """
SELECT ?node (COUNT(?any) AS ?degree)
WHERE {
  {
    { ?node ?p1 ?any }     # Outgoing edges
    UNION
    { ?any ?p2 ?node }     # Incoming edges
  }
}
GROUP BY ?node
"""

results = send_query(query_degrees, sparql, JSON)

degrees = []
for binding in results["results"]["bindings"]:
    node = binding["node"]["value"]
    degree = int(binding["degree"]["value"])
    degrees.append((node, degree))
    debug_print(node + ": " + str(degree))

# Summe aller Degrees
sum_of_degrees = sum(d for _, d in degrees)

# sum of degrees should be equal to 2 * nE
print("Sum of Degrees: " + str(sum_of_degrees))

if nG > 0:
    mean_degree = (2 * nE) / nG
    squared_diffs = [(deg_v - mean_degree) ** 2 for _,deg_v in degrees]
    degree_distribution = sum(squared_diffs) / (nG-1)
else:
    degree_distribution = 0 

print("FINISHED CALCULATING DEGREE DISTRIBUTION")

print("\nRESULT:")
print("-Degree Distribution: " + str(degree_distribution))

STARTING CALCULATING DEGREE DISTRIBUTION
Number of edges in graph:  118
Number of nodes in graph:  78
b0: 4
An ro:SemanticAnnotation is a specialisation of ao:Annotation which requires that ao:body points to an RDF Graph.

This might be a Named Graph or a resource which can be resolved separately from the URI given by ao:body.

This graph SHOULD mention the resources identified by ao:annotatesResource from this annotation, preferably by using their URIs as subject or object of statements.

Note that this use of ao:body is distinct from ao:hasTopic, which also allows the association of a an RDF Graph with an ao:Annotation, but which also implies that this graph is the "topic" (subproperty of bookmark:hasTopic) of the annotated resource. This class does not require this interpretation, it is merely enough that the annotation body mentions the annotated resource, for instance to give it a dc:title or to relate two annotated resources.  Also note that the next version of the AO ontology (v

In [5]:
print("STARTING CALCULATING ENTITIES/CLASSES/PROPERTIES/...")

# TODO: Entities

# Number of instances per type
query_inst = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?type (COUNT(?s) AS ?count)
WHERE {
  ?s rdf:type ?type . 
}
GROUP BY ?type """

results = send_query(query_inst, sparql, JSON)
print(results)
num_instances = 0

for binding in results["results"]["bindings"]:
        rdf_type = binding["type"]["value"]
        count = int(binding["count"]["value"])
        num_instances += count
        
# Number of classes
# Defintion of Class: 
# source: 213, page: 5 - source 250, page 3
# TNOC (total number of classes/concepts) = classes, subclasses, superclasses, anonymous classes
# anonymous classes = equivalent/restriction/unionOf/intersectionOf/complementOf/oneOf/hasValue classes

query_classes = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?class
WHERE {
  {
    # 1. explicitly/implicitly used RDF classes
    # explicitly: ?class a owl:Class . or ?class a rdfs:Class .
    # implicitly: ?any rdf:type ?class . (includes also explicitly used classes)

    ?any rdf:type ?class .
  }
  UNION
  {
    # 2. subclasses
    ?class rdfs:subClassOf ?any .
  }
  UNION
  {
    # 3. superclasses
    ?any rdfs:subClassOf ?class .
  }
  UNION
  {
    # 4. classes used with owl:equivalentClass
    { ?class owl:equivalentClass ?any . }
    UNION
    { ?any owl:equivalentClass ?class . }
  }
  UNION
  {
    # 5. OWL restriction classes
    ?class a owl:Restriction .
  }
  UNION
  {
    # 6. complex classes with using unionOf, intersectionOf etc.
    ?class owl:unionOf|owl:intersectionOf|owl:complementOf|owl:oneOf ?list .
  }
  UNION
  {
    # 7. OWL hasValue restrictions
    ?class owl:hasValue ?val .
  }
}
"""

results = send_query(query_classes, sparql, JSON)

num_classes = 0

for binding in results["results"]["bindings"]:
        rdf_class = binding["class"]["value"]
        num_classes += 1
        debug_print("Class " + str(num_classes) + ": " + str(rdf_class))

# number of properties in T-Box
query_properties = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (COUNT(DISTINCT ?property) AS ?NoP)
WHERE {
  VALUES ?type { owl:ObjectProperty owl:DatatypeProperty owl:AnnotationProperty }
  ?property rdf:type ?type .
}
"""

results = send_query(query_properties, sparql, JSON)

num_properties = 0

print(results)

for binding in results["results"]["bindings"]:
  num_properties = int(binding["NoP"]["value"])

# Number of object properties
# Non-Inheritance -> excluding inheritance properties like rdfs:subPropertyOf or rdfs:subClassOf
query_object_properties = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(DISTINCT ?property) AS ?numObjectProperties)
WHERE {
  ?property rdf:type owl:ObjectProperty .
}
"""

num_obj_properties = 0

results = send_query(query_object_properties, sparql, JSON)

for binding in results["results"]["bindings"]:
  num_obj_properties = int(binding["numObjectProperties"]["value"])

print("RESULTS: ")
print("-Number of properties: " + str(num_properties))
print("-Number of classes: " + str(num_classes))
print("-Number of instances: " + str(num_instances))
print("-Number of object properties: " + str(num_obj_properties))
        

STARTING CALCULATING ENTITIES/CLASSES/PROPERTIES/...
{'head': {'vars': ['type', 'count']}, 'results': {'bindings': [{'type': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#FunctionalProperty'}, 'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'typed-literal', 'value': '1'}}, {'type': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#ObjectProperty'}, 'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'typed-literal', 'value': '1'}}, {'type': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Restriction'}, 'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'typed-literal', 'value': '14'}}, {'type': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#DatatypeProperty'}, 'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'typed-literal', 'value': '1'}}, {'type': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Ontology'}, 'count': {'datatype': 'http://www.w3.org/20

In [ ]:
# TODO: Depth of Inheritance Tree
# Calculating all paths from one root node (start node) for a sparql endpoint



def find_all_paths_subclasses(endpoint_url, default_graph,  start_node):
    neighbors_cache = {}  # global oder in der Funktion definiert

    # helperfunction for finding neighbors of a node (triple=(node, pred, neighbor))
    def get_neighbors(node):
        
        node_str = str(node)

        # 1. Prüfen, ob der Node bereits im Cache ist
        if node_str in neighbors_cache:
            # Debug-Ausgabe
            #print(f"Cache-Treffer für {node_str}")
            return neighbors_cache[node_str]
 
        if (len(str(node)) == 0): #or (not (node.startswith("http://") or node.startswith("https://"))):
            return []  # Literale und Blank Nodes haben keine gültigen SPARQL-Patterns

        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?child
        WHERE {{
          ?child rdfs:subClassOf <{node}> .
        }}
        """
        
        #sparql.setQuery(query)
        #sparql.setReturnFormat(JSON)
        sparql.addDefaultGraph(default_graph)
        results = send_query(query, sparql, JSON)
       
        # Alle Objekte der Tripel extrahieren
        neighbors = []
        for binding in results["results"]["bindings"]:
            next_obj = binding["next"]
            value = next_obj["value"]
            value_type = next_obj["type"]  # 'uri', 'literal', 'bnode'
            #if value.startswith("POLYGO"):
             #   print("Value Type: " + value_type)

            """if value_type == "literal" or value_type == "typed-literal":
                lang_tag = next_obj.get("xml:lang")
                datatype = next_obj.get("datatype")
                
                if lang_tag:  # Sprach-Tag hinzufügen
                    #if value.startswith("POLYGO"):
                    #    print("lang_tag: " + lang_tag)
                    value = f"{value}@{lang_tag}"
                elif datatype:  # Datentyp hinzufügen (optional, für Vollständigkeit)
                   # if value.startswith("POLYGO"):
                    #    print("datatype: " + datatype)
                    value = f"{value}^^{datatype}"
            
            if value_type != "literal" and value_type != "typed-literal" and value_type != "uri" and value_type != "iri" and value_type != "bnode":
                print("OTHER VALUE TYPE: " + value_type)"""

            neighbors.append((value, value_type))
            #neighbors.append(binding["next"]["value"])
            #if str(node) == "https://nfdi.fiz-karlsruhe.de/ontology/MIME-Type":
             #   value_type = binding["next"]["type"]  # <--- Hier bekommst du den Typ: uri, literal, bnode
              #  value = binding["next"]["value"]
               # # Debug-Output
                #print(f"  Nachbar gefunden: {value} (Typ: {value_type})")    

        neighbors_cache[node_str] = neighbors

        return neighbors
    
    def dfs(path, node, node_type="uri"):
        global num_paths_inh_tree
        global max_depth_inh_tree

        # skip node if it already is in path (avoiding cycles)
        if node in path:
            return
        
        # add current node to path
        path.append(node)
        
        neighbors = list(get_neighbors(node)) 

        if not neighbors:
            paths.append(list(path))
            num_paths_inh_tree += 1
            max_depth_inh_tree = max(max_depth_inh_tree, len(path) - 1)
        else:
            for neighbor, neighbor_type in neighbors:
              #print("\node: " + str(node) + " - neighbor: " + str(neighbor))
              # if node is literal -> it does not have any neighbors -> path is finished
              #if neighbor_type == "literal" or (neighbor in literals):
              """if neighbor in literals:
                  #print("Ist in Literals: " +  neighbor)
              #if neighbor in literals:
                  path.append(neighbor)
                  #print("Found Path: ", path)
                  paths.append(list(path))
                  num_paths += 1
                  abs_depth += len(path) - 1
                  max_depth = max(max_depth, len(path) - 1)
                  # remove node from the path to find next path
                  path.pop()
              else:"""
              dfs(path, neighbor, neighbor_type)

        # remove node from the path to find next path
        path.pop()

    sparql = SPARQLWrapper(endpoint_url)
    sparql.addDefaultGraph(default_graph)
    #sparql.setTimeout(60)  # 120 Sekunden Timeout

    # list which stores all paths
    paths = []

    # TODO: ich glaub dass node_type=uri nicht passt, weil bnodes auch vorkommen können (vllt passts aber auch mt uri weil gleich gesucht wird)
    dfs([], start_node, node_type="uri")

    return paths

#endpoint_url = ""
default_graph = "http://def.seegrid.csiro.au/ontology/om/sam-lite"
sparql = SPARQLWrapper(endpoint_url)
sparql.addDefaultGraph(default_graph)
# ?root a owl:Class -> to ensure root is a class
# FILTER NOT EXISTS {?root rdfs:subClassOf ?anyClass .} -> to get root which has no superclass
query_root = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?root
WHERE {
  {
    ?root rdf:type owl:Class .
  }
  UNION
  {
    ?root rdf:type rdfs:Class .
  }
  
  FILTER NOT EXISTS {
    ?root rdfs:subClassOf ?anyClass .
  }
}
"""

# node types that can be in results: uri/iri or blank nodes  
results = send_query(query_root, sparql, JSON)

tree_roots = set()

for binding in results["results"]["bindings"]:
    node = binding["root"]["value"]
    debug_print("Found Tree node: " + node)
    tree_roots.add(node)

# Anzahl der Root-Nodes ausgeben
debug_print(f"Number of root nodes: {len(tree_roots)}")
    
max_depth_inh_tree = 0
num_paths_inh_tree = 0

# Dictionary für alle Pfade
all_paths = {}

# Jetzt alle Pfade für die gespeicherten Root-Nodes berechnen
for root in tree_roots:
    debug_print("Calculating subclass paths for tree node: " + root)
    all_paths[root] = find_all_paths_subclasses(endpoint_url, default_graph, root)

# Output found paths
print("FOUND PATHS:")
for root_node, paths in all_paths.items():
    for path in paths:
        debug_print(path)
        debug_print("Path length = " + str(len(path)-1))

debug_print("-Number of Paths: " + str(num_paths_inh_tree))
print("-Depth of Inheritance Tree: " + str(max_depth_inh_tree))

# Problem: Ich glaube in diesem dataset ist die tiefe einfach 0 (siehe chatgpt)

Found Tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#ObservationProcess
Found Tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Process
Found Tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Measure
Found Tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Location
Found Tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#SampleTransformationAgent
Found Tree node: b0
Number of root nodes: 6
Calculating subclass paths for tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Process
Calculating subclass paths for tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Location
Calculating subclass paths for tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#SampleTransformationAgent
Calculating subclass paths for tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#Measure
Calculating subclass paths for tree node: http://def.seegrid.csiro.au/ontology/om/sam-lite#ObservationProcess


KeyboardInterrupt: 

In [7]:
# Property Class Ratio
if num_classes > 0:
    prop_class_ratio = num_properties / num_classes 
    print("Property Class Ratio: " + str(prop_class_ratio))
else:
    # TODO
    print("Property Class Ratio is INF!")

# Class Property Ratio
if num_properties > 0:
    class_prop_ratio = num_classes / num_properties 
    print("Class Property Ratio: " + str(class_prop_ratio))
else:
    # TODO
    print("Class Property Ratio is INF!")

# Inheritance Richness = average number of subclasses per clas (source 227 - page 9) 
query_subclasses = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (COUNT(*) AS ?numInheritanceRelations)
WHERE {
  ?subclass rdfs:subClassOf ?superclass .
}
"""

results = send_query(query_subclasses, sparql, JSON)

for binding in results["results"]["bindings"]:
  num_subclasses = int(binding["numInheritanceRelations"]["value"])

if num_classes > 0:
    inheritance_richness = num_subclasses / num_classes 
    print("Inheritance Richness: " + str(inheritance_richness))
else:
    # TODO
    print("Inheritance Richness is INF!")

query_datatype_properties = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (COUNT(DISTINCT ?property) AS ?numDatatypeProperties)
WHERE {
  ?property rdf:type owl:DatatypeProperty .
}
"""

results = send_query(query_datatype_properties, sparql, JSON)

for binding in results["results"]["bindings"]:
  num_datatype_properties = int(binding["numDatatypeProperties"]["value"])

debug_print("Number of Datatype properties: " + str(num_datatype_properties))

if num_classes > 0:
    attr_richness = num_datatype_properties / num_classes
    print("Attribute Richness: " + str(attr_richness))
else:
    print("Attribute Richness is INF!")


Property Class Ratio: 0.06060606060606061
Class Property Ratio: 16.5
Inheritance Richness: 1.0303030303030303
Number of Datatype properties: 0
Attribute Richness: 0.0


In [8]:
# Average Connectivity
if num_classes > 0:
    avg_population = num_instances / num_classes
    print("Average Population: " + str(avg_population))
else:
    print("Average Population is INF!")


Average Population: 0.7878787878787878


In [9]:
# TODO: Cohesion

DEBUG = True


def format_literal_for_sparql(value: str) -> str:
    # Escape Backslashes und Quotes
    safe_val = value.replace("\\", "\\\\").replace('"', '\\"')
    # Ersetze echte Zeilenumbrüche mit \n
    safe_val = safe_val.replace("\n", "\\n")
    
    if "@en" in value or value.count("@") == 1:
        val, lang = value.rsplit("@", 1)
        safe_val = val.replace("\n", "\\n").replace('"', '\\"')
        return f'"{safe_val}"@{lang}'
    elif "^^" in value:
        val, dtype = value.split("^^", 1)
        safe_val = val.replace("\n", "\\n").replace('"', '\\"')
        return f'"{safe_val}"^^<{dtype}>'
    else:
        return f'"{safe_val}"'


def get_all_neighbors(node):
  #debug_print(node[0])
  #debug_print(node[1])

  # node[0] = node_obj["value"]
  # node[1] = node_obj["type"]
  #query = f"""
  #  SELECT DISTINCT ?neighbor WHERE {{
  #      {{ <{node[0]}> ?p1 ?neighbor . }}
   #     UNION 
   #     {{ ?neighbor ?p2 <{node[0]}> . }}
  #  }}
  #  """
  
  value, node_type = node

  if node_type in ["literal", "typed-literal"]:
    literal = format_literal_for_sparql(value)
    # Literal kann nur Objekt sein
    # SPARQL-Syntax für Literal: mit Quotes und optionalem Datentyp oder Lang-Tag
    """if "@" in value:
        literal = f'"{value.split("@")[0]}"@{value.split("@")[1]}'
    elif "^^" in value:
        lit_val, dtype = value.split("^^")
        literal = f'"{lit_val}"^^<{dtype}>'
    else:
        literal = f'"{value}"'"""

    query = f"""
    SELECT DISTINCT ?neighbor WHERE {{
        ?neighbor ?p2 {literal} .
    }}
    """
    print("\nLITERAL QUEUE:")
    print(query)

  else:
    query = f"""
    SELECT DISTINCT ?neighbor WHERE {{
        {{ <{value}> ?p1 ?neighbor . }}
        UNION
        {{ ?neighbor ?p2 <{value}> . }}
    }}
    """

    #print("\n QUEUE:")
    #print(query)

  results = send_query(query, sparql, JSON)

  return results

def check_literal_tags(node_obj, node, node_type):
  if node_type == "literal" or node_type == "typed-literal":
    lang_tag = node_obj.get("xml:lang")
    datatype = node_obj.get("datatype")
    
    if lang_tag:  # add language tag
        node = f"{node}@{lang_tag}"
    elif datatype:  # add datatype tag
        node = f"{node}^^{datatype}"
    
  return node

# Getting all nodes

query_all_nodes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?node WHERE {
    { ?node ?p ?o } UNION { ?s ?p ?node }
}
"""
results = send_query(query_all_nodes, sparql, JSON)

all_nodes = set()

for binding in results["results"]["bindings"]:
  node_obj = binding["node"]
  node = node_obj["value"]
  node_type = node_obj["type"]

  # if node is of type literal or typed-literal -> tags are being added
  node = check_literal_tags(node_obj, node, node_type)

  all_nodes.add((node, node_type))      

for node, node_type in all_nodes:
  # repr -> shows string as python code (--> new line = \n, etc.)
  debug_print(f"[{node_type}] {repr(node)}")

## algorithm: searches for components of graph

# describes number of independent components of graph (like subgraphs which are connected internally)
components = 0

# describes visited / discovered nodes
visited = set()

while visited != all_nodes:
  
  # Choose first unvisited node
  # iter creates iterator for set - next gives next element in set
  start_node = next(iter(all_nodes - visited))
  frontier = {start_node}
  visited.add(start_node)

  while frontier:
    new_frontier = set()
    for node in frontier:
      #print(node)
      if node[1] in ["literal", "typed-literal"]:
        debug_print("Searching for neighbors of: " + str(node))
      results = get_all_neighbors(node)

      neighbors = set()

      for binding in results["results"]["bindings"]:
        neighbor_obj = binding["neighbor"]
        neighbor = neighbor_obj["value"]
        neighbor_type = neighbor_obj["type"]

        # if node is of type literal or typed-literal -> tags are being added
        neighbor = check_literal_tags(neighbor_obj, neighbor, neighbor_type)
        if node[1] in ["literal", "typed-literal"]:
          debug_print("Found neighbor: " + neighbor)

        neighbors.add((neighbor, neighbor_type))
        visited.add((neighbor, neighbor_type))

      new_nodes = {n for n in neighbors if n in all_nodes and n not in visited}

      new_frontier |= new_nodes
    
    frontier = new_frontier
      #print("Visited Components: " + len(visited))

  components += 1

  print(f"Komponente {components} mit Startknoten {start_node[0]} abgeschlossen. Besucht: {len(visited)}/{len(all_nodes)}")

print("Cohesion: " + str(components))

# TODO: Solve following problems:
# LITERAL QUEUE:  SELECT DISTINCT ?neighbor WHERE { ?neighbor ?p2 ""download""@en .} does not work
# addressing bnodes with <bnode-id> does not work for every endpoints
# Literals with \n and emtpy spaces do not work in neighbor search! (ich denke das ist gelöst durch format_literal_for_sparql)

[typed-literal] 'The class OM_Process (Figure 2) is an instance of the «metaclass» GF_FeatureType (ISO 19109:2005), which therefore represents a feature type. OM_Process is abstract, and has no attributes, operations or associations. It serves as the base class for observation processes. The purpose of an observation process is to generate an observation result. An instance of OM_Process is often an instrument or sensor, but may be a human observer, a simulator, or a process or algorithm applied to more primitive results used as inputs. \r\n\r\nNOTE\tISO 19115-2:2008 provides MI_Instrument, LE_Processing and LE_Algorithm, which could all be modelled as specializations of OM_Process. OGC SensorML [10] provides a model which is suitable for many observation procedures.^^http://www.w3.org/2001/XMLSchema#string'
[uri] 'http://def.seegrid.csiro.au/isotc211/iso19123/2005/coverage#DiscreteCoverage'
[typed-literal] 'Copyright © 2012-2013 CSIRO^^http://www.w3.org/2001/XMLSchema#string'
[literal

c:\Users\guene\OneDrive\Studium\6. Semester\Bachelorarbeit\Obsidian\code\venv\Lib\site-packages\SPARQLWrapper\Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/plain;charset=UTF-8' returning raw response...
  warnings.warn(


TypeError: byte indices must be integers or slices, not str

In [ ]:
# Average Class Connectivity
# Connectivity of a class is defined as the total number of relationships instances of 
# the class have with instances of other classes (source 227 - page 10)

# looking for number of triples (c1, p, c2) or (c3, p, c1) for each class with instances c1
# c1, c2 are instances of classes 
# c1 != c2
# property != rdf:type because we are not interested in the class relationships 
query_class_connectivity = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT ?class (COUNT(*) AS ?connectivity)
WHERE {
  {
    ?instance ?property ?target .

    ?instance rdf:type ?class .
    ?target rdf:type ?targetClass .

    FILTER(?property != rdf:type)
    FILTER(?class != ?targetClass)
  }
  UNION
  {
    ?instance ?property ?target .

    ?target rdf:type ?class .
    ?instance rdf:type ?targetClass .

    FILTER(?property != rdf:type)
    FILTER(?class != ?targetClass)
  }
}
GROUP BY ?class

"""
results = send_query(query_class_connectivity, sparql, JSON)

class_connectivity_list = []
sum_connectivities = 0

for binding in results["results"]["bindings"]:
    class_name = binding["class"]["value"]
    connectivity = int(binding["connectivity"]["value"])
    class_connectivity_list.append((class_name, connectivity))
    sum_connectivities += connectivity


for class_name, connectivity in class_connectivity_list:
  debug_print("Connectivity of Class " + class_name + ": " + str(connectivity))

if num_classes > 0:
  avg_class_connectivity = sum_connectivities / num_classes
  print("Average Class Connectivity: " + str(avg_class_connectivity))
else:
  print("Average Class Connectivity is INF!")


In [ ]:
import requests

endpoint_url = "https://data.europa.eu/sparql"
#endpoint_url = "https://lov.linkeddata.es/dataset/lov/sparql"
endpoint_url = "https://sparql.europeana.eu/"
#endpoint_url = "https://nfdi4culture.de/sparql"
endpoint_url = "https://query.wikidata.org/sparql"
#endpoint_url = "https://graphdb.ontotext.com/repositories/ontotext_public?query=SELECT+*+WHERE+%7B%7D+LIMIT+1"
#endpoint_url = "https://sparql.bioontology.org/"
#endpoint_url = "http://rdf4j.org/sparql"
#endpoint_url = "https://franz.com/agraph/sparql"

r = requests.get(endpoint_url)
print(r.headers.get("Server"))
print(r.text[:1000])  # manchmal im HTML ein Hinweis wie "Virtuoso" oder "Fuseki"
